In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import TimeSeriesSplit

In [4]:
a = range(0,34)

In [16]:
n_splits = 4
for i in range(n_splits, a[-1]):
    print 'Train : [0 {}], Test: {}'.format(i, i+1)
    i += 1

Train : [0 4], Test: 5
Train : [0 5], Test: 6
Train : [0 6], Test: 7
Train : [0 7], Test: 8
Train : [0 8], Test: 9
Train : [0 9], Test: 10
Train : [0 10], Test: 11
Train : [0 11], Test: 12
Train : [0 12], Test: 13
Train : [0 13], Test: 14
Train : [0 14], Test: 15
Train : [0 15], Test: 16
Train : [0 16], Test: 17
Train : [0 17], Test: 18
Train : [0 18], Test: 19
Train : [0 19], Test: 20
Train : [0 20], Test: 21
Train : [0 21], Test: 22
Train : [0 22], Test: 23
Train : [0 23], Test: 24
Train : [0 24], Test: 25
Train : [0 25], Test: 26
Train : [0 26], Test: 27
Train : [0 27], Test: 28
Train : [0 28], Test: 29
Train : [0 29], Test: 30
Train : [0 30], Test: 31
Train : [0 31], Test: 32
Train : [0 32], Test: 33
